In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qa-main/val_data.csv
/kaggle/input/qa-main/train_data.csv
/kaggle/input/fin-qa-testing/Q_test_data.csv


In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
train_path ='/kaggle/input/qa-main/train_data.csv'
eval_path = '/kaggle/input/qa-main/val_data.csv'

In [4]:
train_data = pd.read_csv(train_path)
eval_data = pd.read_csv(eval_path)

In [5]:
train_data.shape
eval_data.shape

(96, 5)

In [6]:
train_data.head()

,Unnamed: 0,Q,A,MQ,MA
0,36,how error minimization is done ? Is there any ...,Error minimization in machine learning involve...,Elaborate on error minimization techniques in ...,"In machine learning error minimization, techni..."
1,49,Dealing with missing data in a dataset?,"Imputation methods, like mean or median imputa...",How can missing values in a dataset be address...,"Imputation, deletion, ignoring (depending on t..."
2,184,Which performance metric is better R2 or adjus...,Adjusted R2 because the performance of predict...,"Compared to R2, why is adjusted R2 a better p...",Adjusted R2 is more reliable since it conside...
3,53,What part of BERT's architecture gives it Bidi...,"The Transformer Encoder, which is a Bidirectio...",Which component of BERT's architecture provide...,Masked language model and next sentence predic...
4,291,"Suppose, if I have in channels= 3 and out chan...","Yes, that's correct. In the context of a convo...","I have in channels = 3 and out channels = 16, ...","In a Convolutional Neural Network (CNN), the i..."


In [7]:
eval_data.head()

,Unnamed: 0,Q,A,MQ,MA
0,106,"What is the concept of ""function approximation...",Function approximation in Reinforcement Learni...,Reinforcement Learning: Explain the idea of us...,Function approximation in Reinforcement Learni...
1,53,Role of ML in artificial intelligence?,ML is a subset of AI; it empowers AI systems t...,How does Machine Learning contribute to the br...,ML is a subfield of AI focused on learning fro...
2,185,What do you mean by AUC curve?,AUC (area under curve). Higher the area under ...,Describe AUC curve in detail.,Area Under Curve (AUC) refers to the measure o...
3,58,Is it The activation function output of the ne...,The activation function output of a neuron is ...,Confirm if the activation function outputs the...,Activation function output results from applyi...
4,63,When our test accuracy is lesser than train ac...,"Yes, a lower test accuracy compared to train a...",Connect overfitting with lower test accuracy a...,"Yes, when test accuracy is lower than train ac..."


In [8]:
# data.columns

In [9]:
# train_data=data.sample(frac=0.8,random_state=42)
# eval_data=data.drop(train_data.index)

In [10]:
# !rm -rf /kaggle/working/QA_Train.txt

In [11]:
train_data = train_data.drop('Unnamed: 0', axis=1)
eval_data = eval_data.drop('Unnamed: 0', axis=1)

In [12]:
with open('/kaggle/working/QA_Test.txt', 'w') as file:
    for index, row in eval_data.iterrows():
        file.write("Question1 : " + f"{row['Q']}" + " Answer1 : " + f"{row['A']}" + "Question2 : " + f"{row['MQ']}" + "Answer2 : " + f"{row['MA']}\n")
file.close()

In [13]:
with open('/kaggle/working/QA_Train.txt', 'w') as file:
    for index, row in train_data.iterrows():
        file.write("Question1 : " + f"{row['Q']}" + " Answer1 : " + f"{row['A']}" + "Question2 : " + f"{row['MQ']}" + "Answer2 : " + f"{row['MA']}\n")
file.close()

In [14]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers accelerate evaluate rouge_score

Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.4 MB/s eta 0:00:00
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
   ━━━━

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
!pip install --upgrade tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e3/ba/aa8a76eff5c20761b0361a5b4c9fccb8742c29a82adba7a8ad8ae819984e/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for ml-dtypes~=0.2.0 from https://files.pythonhosted.org/packages/d1/1d/d5cf76e5e40f69dbd273036e3172ae4a614577cb141673427b80cac948df/ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.pythonhosted.org/packages/6e/0c/1059a6682cf2cc1fcc0d5327837b5672fe4f5574255fa5430d0a8ceb75e9/tensorboard-2.15.1-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow-estimator<2.16,>=2.15.0 from https://files.pythonhosted.org/packages/b6/c8/2f823c8958d5342eafc6dd3e922f0cc4fcf8c2e0460284cc462dae3b60a0/tensorfl

In [17]:
!pip install sacrebleu

  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/de/ea/025db0a39337b63d4728a900d262c39c3029b0fe76a9876ce6297b1aa6a0/sacrebleu-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 154.0 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 664.9 kB/s eta 0:00:00a 0:00:01


In [18]:
import evaluate
sacrebleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load('rouge')

In [19]:
# File paths

train_path = "/kaggle/working/QA_Train.txt"
test_path = "/kaggle/working/QA_Test.txt"
output_dir = '/kaggle/working/output/'

In [20]:
# !rm -rf /kaggle/working/wandb


In [21]:
model_name = 'gpt2'
overwrite_output_dir = False
per_device_train_batch_size = 1
num_train_epochs = 12
save_steps = 1000

In [22]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm
    )
    return data_collator

In [23]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def preprocess_logits_for_metrics(logits, labels):
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

In [24]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds

    # print(type(logits))
    # print(len(logits))
    # print('logits[0].shape:', logits[0].shape)
    # print('logits[0]:', logits[0])
    # print('logits[1].shape:', logits[1].shape)
    # print('logits[1]:', logits[1])
    # print('labels.shape:', labels.shape)
    # print('labels:', labels)

    preds = logits[0]
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    preds, labels = postprocess_text(decoded_preds, decoded_labels)

    # print(len(preds), len(labels))
    # print(preds[0])
    # print(labels[0])

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    blue_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels,lowercase = True)

    # print(result)

    return {
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(blue_result["score"], 4)
    }

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.save_pretrained(output_dir)

def load_dataset(file_path, tokenizer):
    dataset = LineByLineTextDataset(
                tokenizer=tokenizer,
                file_path=file_path,
                block_size=512
    )

    return dataset

train_dataset = load_dataset(train_path, tokenizer)
test_dataset = load_dataset(test_path, tokenizer)


data_collator = load_data_collator(tokenizer)

tokenizer.save_pretrained(output_dir)

training_args = TrainingArguments(
          output_dir=output_dir,
          evaluation_strategy = "steps",
          eval_steps = 1000,
          learning_rate=1e-5,
          save_strategy = "epoch",
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          per_device_eval_batch_size=1,
          num_train_epochs=num_train_epochs
      )

trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=test_dataset,
          preprocess_logits_for_metrics=preprocess_logits_for_metrics,
          compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model()

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,R1,R2,Rl,Rlsum,Bleu
1000,2.997400,3.197643,0.474700,0.199200,0.368900,0.370500,24.515500
2000,2.728800,3.054548,0.480900,0.201500,0.376800,0.378000,24.915700
3000,2.471000,3.029690,0.482800,0.207500,0.383100,0.384400,25.501600
4000,2.322900,2.997269,0.483200,0.207900,0.384400,0.385400,25.528000
5000,2.239000,2.987695,0.487200,0.208300,0.385700,0.386800,25.705800
6000,2.086100,2.993351,0.487100,0.207500,0.384300,0.386000,25.660000
7000,2.038700,3.007855,0.485700,0.208300,0.386800,0.387900,25.710100
8000,1.981800,3.027161,0.482200,0.207300,0.385200,0.386200,25.740700
9000,1.977100,3.035964,0.481700,0.205500,0.382400,0.383900,25.450300
10000,1.917600,3.037876,0.483800,0.207800,0.385700,0.387100,25.753800


In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_gpt2_model(model_path):
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = GPT2LMHeadModel.from_pretrained(model_path)

    return model, tokenizer

# Replace 'path/to/your/gpt2/model' with the actual path where your GPT-2 model is saved
model_path = '/kaggle/working/output'
gpt2_model, gpt2_tokenizer = load_gpt2_model(model_path)




def generate_output_gpt2(model, tokenizer, question, temperature=1.0, top_p=0.9, max_length=50):
    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors="pt")

    # Generate the model output with temperature and top-p sampling
    model_output = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_p=top_p,
        top_k=50,
    )

    # Decode the generated output
    generated_output = tokenizer.decode(model_output[0], skip_special_tokens=True)

    return generated_output

# Example question with temperature=0.7 and top-p=0.9
input_question = "What NLP?"

# Get the generated output
output = generate_output_gpt2(gpt2_model, gpt2_tokenizer, input_question, temperature=0.7, top_p=0.9)

# Print or use the generated output
print(output)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


What NLP?  It's a subset of machine learning, where the goal is to learn from data and apply it to new problems.  It's a subset of machine learning that focuses on problem-solving and problem-solving without the


In [27]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


In [28]:
def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


In [30]:
def generate_text(sequence, max_new_tokens):
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    input_length = ids.size(1)
    max_length = input_length + max_new_tokens
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)




model_path = '/kaggle/working/output'
model = load_model(model_path)
tokenizer = load_tokenizer(model_path)


def root(prompt: str):
    result = generate_text("Question: " + prompt + "Answer: ", 35).split('Answer: ')[1]
    try:
        result = result.split('.')[0] + '.'
    except Exception as e:
        print(e)
    return result

root('what is K-mean algorithm?')

' In K-Means,  the k-mean component is the median value (in terms of importance) of the predicted value in the class.'

In [29]:
import pandas as pd

# def load_model(model_path):
#     model = GPT2LMHeadModel.from_pretrained(model_path)
#     return model

# def load_tokenizer(tokenizer_path):
#     tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
#     return tokenizer

def generate_text(sequence, max_new_tokens):
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    input_length = ids.size(1)
    max_length = input_length + max_new_tokens
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)




In [37]:
df = pd.read_csv(csv_path)

In [38]:
df.dtypes

Q    object
dtype: object

In [30]:
def generate_answers_from_csv(csv_path, max_new_tokens):
    df = pd.read_csv(csv_path)
    df['Q'] = df['Q'].astype(str)
    generated_answers = []

    for index, row in df.iterrows():
        question = row['Q']  # Adjust column name as per your CSV
        answer = generate_text("Question: " + question + "Answer: ", max_new_tokens).split('Answer: ')[1]

        try:
            answer = answer.split('.')[0] + '.'
        except Exception as e:
            print(e)

        generated_answers.append({'Question': question, 'Answer': answer})

    return generated_answers



In [31]:
model_path = '/kaggle/working/output'
model = load_model(model_path)
tokenizer = load_tokenizer(model_path)

csv_path = '/kaggle/input/fin-qa-testing/Q_test_data.csv'
max_new_tokens = 35
generated_answers = generate_answers_from_csv(csv_path, max_new_tokens)



# Create a DataFrame from the list of dictionaries
df_generated_answers = pd.DataFrame(generated_answers)

# Save the DataFrame to a CSV file
output_csv_path = '/kaggle/working/generated_answers.csv'
df_generated_answers.to_csv(output_csv_path, index=False)

print("Generated answers saved to:", output_csv_path)

Generated answers saved to: /kaggle/working/generated_answers.csv


In [32]:
for item in generated_answers:
    print(f"Question: {item['Question']}\nAnswer: {item['Answer']}\n")

Question: Can an SVM use gradient descent to maximize its margin?
Answer:  Gradient descent has several advantages over traditional deep learning, including avoiding overfitting, maximizing accuracy.

Question: How to deal with the curse of dimensionality?
Answer:  Dimensionality is a generalization problem, as it entails taking dimensions that are not consistent with the data dimensions.

Question: Importance of bias-variance tradeoff in ML?
Answer:  Bias-variance is often associated with generalization error, but not generalization error.

Question: What is the main contribution of the paper "Tree-structured regional CNN-LSTM model for dimensional sentiment analysis" by Wang et al.?
Answer:  This paper proposes unsupervised global neural networks designed to incorporate feature-driven model learning, enabling deep sentiment analysis.

Question: Can KNN be used for regression tasks?
Answer:  Logistic Regression models include supervised learning (M), supervised learning (S), and regre

In [ ]:
kaggle kernels pull tany14/qa-gpt-own-data